In [13]:
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from transformers.pipelines import pipeline
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

# 加载数据

In [2]:
# step1 加载文件
with open('../../data/切词.txt', 'r', encoding='utf-8') as file:
  docs = file.readlines()
print('条数: ', len(docs))
print('预览第一条: ', docs[0])

vectorizer_model = None

条数:  1000
预览第一条:  文旅文 创看 洛阳 河南省 文旅文创 发展 大会 本次 大会 安排 项目 签约 主要 方面 内容 一是 文旅 产业 项目 签约 截至 目前 梳理 重点 文旅 项目 投资总额 525.6 亿元 遴选 重大项目 进行 现场 签约 投资总额 365.8 亿元 项目 包括 文物 数字化 开发 文化 创意 园区 建设 文化 项目 涵盖 旅游 度假区 建设 旅游 酒店 民宿 打造 旅游 项目 既有 旅游 景区 开发 商旅 综合体 建设 传统 业态 项目 宇宙 基地 沉浸 演艺 业态 项目 充分体现 我省 文化 旅游 发展 特点 趋势 二是 引客 入豫 项目 签约 主要 我省 文旅 部门 文旅 企业 头部 旅行 知名 OTA 平台 重点 客源地 文旅 部门 签订 引客 入豫 协议 持续 拓展 省外 客源 市场



# 使用BERT-Base-Chinese

In [3]:
# 1. 词向量模型，同时加载本地训练好的词向量
embedding_model = pipeline("feature-extraction", model="bert-base-chinese") # 使用bert-base-chinese
embeddings = np.load('../../data/embedding_bbc.npy') # 使用bert-base-chinese向量
print(embeddings.shape)

# 2. 创建分词模型
vectorizer_model = CountVectorizer() # 因为我们已经分好词了，所以这里不需要传入分词函数了

# 3. 创建UMAP降维模型
umap_model = UMAP(
  n_neighbors=15,
  n_components=5,
  min_dist=0.0,
  metric='cosine',
  random_state=42  # ⚠️ 防止随机 https://maartengr.github.io/BERTopic/faq.html
)

# 4. 创建HDBSCAN聚类模型
# 如果要建设离群值，可以减小下面两个参数
# https://hdbscan.readthedocs.io/en/latest/faq.html
hdbscan_model = HDBSCAN(
  min_cluster_size=40,
  min_samples=40,
)

# 5. 创建CountVectorizer模型
vectorizer_model = CountVectorizer(stop_words=['洛阳', '旅游', '文化'])

# 正式创建BERTopic模型
topic_model = BERTopic(
  embedding_model=embedding_model,
  vectorizer_model=vectorizer_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
)

# 查看主题
topics, probs = topic_model.fit_transform(docs, embeddings=embeddings) #传入训练好的词向量
topic_model.get_topic_info()

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(1000, 768)


,Topic,Count,Name,Representation,Representative_Docs
0,-1,501,-1_景区_河南_游客_城市,"[景区, 河南, 游客, 城市, 洛阳市, 中国, 历史, 活动, 河南省, 发展]",[河南 多家 景区 陆续 发布 开园 公告 台风 杜苏芮 强度 逐渐 减弱 河南 景区 陆续...
1,0,216,0_中国_石窟_龙门石窟_世界,"[中国, 石窟, 龙门石窟, 世界, 艺术, 旅行, 景区, 位于, 河南, 历史]",[龙门石窟 中国 石刻 艺术 宝库 现为 世界 文化遗产 全国 重点 文物保护 单位 国家 ...
2,1,100,1_活动_景区_免费_高速,"[活动, 景区, 免费, 高速, 门票, 时间, 白云山, 地点, 期间, 体验]",[洛阳 身边 自驾游 栾川 高速 免费 答疑 自驾游 栾川 高速 费全免 问题 需要 了解 ...
3,2,99,2_发展_城市_项目_河南,"[发展, 城市, 项目, 河南, 文旅, 活动, 中国, 国家, 游客, 建设]",[行走 河南 读懂 中国 关注 全省 文旅文创 发展 大会 二十大 报告 指出 坚持 以文塑...
4,3,42,3_地方_很多_一天_时间,"[地方, 很多, 一天, 时间, 已经, 一点, 古城, 酒店, 西安, 感受]",[此次 四天 旅行 已经 结束 以往 不用 这次 大学 同学 一起 毕业 一年 小聚 组团 ...
5,4,42,4_旅客_郑州_高速_景区,"[旅客, 郑州, 高速, 景区, 客流, 出行, 发送, 通行, 假期, 高峰]",[国庆 长假 我市 迎来 客流 高峰 公路 铁路 发送 旅客 超过 万人次 今年 国庆 长假...


In [4]:
# UMAP可视化
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, metric='cosine', min_dist=0.0).fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, hide_document_hover=True, hide_annotations=False)

# 使用哈工大模型

In [8]:
# 1. 词向量模型，同时加载本地训练好的词向量
embedding_model = pipeline("feature-extraction", model="hfl/chinese-bert-wwm") # 使用哈工大模型
embeddings = np.load('../../data/embedding_hfl.npy') # 使用哈工大向量
print(embeddings.shape)

# 2. 创建分词模型
vectorizer_model = CountVectorizer() # 因为我们已经分好词了，所以这里不需要传入分词函数了

# 3. 创建UMAP降维模型
umap_model = UMAP(
  n_neighbors=15,
  n_components=5,
  min_dist=0.0,
  metric='cosine',
  random_state=42  # ⚠️ 防止随机 https://maartengr.github.io/BERTopic/faq.html
)

# 4. 创建HDBSCAN聚类模型
# 如果要建设离群值，可以减小下面两个参数
# https://hdbscan.readthedocs.io/en/latest/faq.html
hdbscan_model = HDBSCAN(
  min_cluster_size=40,
  min_samples=10,
)

# 5. 创建CountVectorizer模型
vectorizer_model = CountVectorizer(stop_words=['洛阳', '旅游', '文化'])

# 正式创建BERTopic模型
topic_model = BERTopic(
  embedding_model=embedding_model,
  vectorizer_model=vectorizer_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
)

# 查看主题
topics, probs = topic_model.fit_transform(docs, embeddings=embeddings) #传入训练好的词向量
topic_model.get_topic_info()

Some weights of the model checkpoint at hfl/chinese-bert-wwm were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(1000, 768)


,Topic,Count,Name,Representation,Representative_Docs
0,-1,353,-1_景区_河南_中国_城市,"[景区, 河南, 中国, 城市, 年票, 洛阳市, 历史, 遗址, 游客, 郑州]",[河南 多家 景区 陆续 发布 开园 公告 台风 杜苏芮 强度 逐渐 减弱 河南 景区 陆续...
1,0,248,0_景区_发展_建设_项目,"[景区, 发展, 建设, 项目, 活动, 郑州, 洛阳市, 国家, 工作, 博物馆]",[好消息 洛阳 再添 4A 景区 洛阳市 文化 旅游局 获悉 洛阳 再添 国家 4A 景区 ...
2,1,111,1_河南_中国_体验_感受,"[河南, 中国, 体验, 感受, 地方, 旅行, 历史, 古城, 打卡, 西安]",[洛阳 值得 一去 地方 简单 下来 洛阳 旅游 攻略 建议 夏天 洛阳 洛阳 市区 主要 ...
3,2,99,2_石窟_龙门石窟_中国_艺术,"[石窟, 龙门石窟, 中国, 艺术, 世界, 莫高窟, 位于, 文化遗产, 造像, 朝代]",[旅行 洛阳 龙门石窟 中国 石刻 艺术 宝库 现为 世界 文化遗产 全国 重点 文物保护 ...
4,3,68,3_活动_景区_免费_高速,"[活动, 景区, 免费, 高速, 门票, 时间, 白云山, 期间, 栾川, 游客]",[洛阳 身边 自驾游 栾川 高速 免费 答疑 自驾游 栾川 高速 费全免 问题 需要 了解 ...
5,4,63,4_游客_景区_城市_五一,"[游客, 景区, 城市, 五一, 热门, 数据, 显示, 增长, 上海, 假期]",[清明 机票价格 回归 至疫前 水平 清明 出游 人数 三成 面对 越来越近 清明 长假 旅...
6,5,58,5_王府_竹海_旅行_景区,"[王府, 竹海, 旅行, 景区, 泉州, 洛阳桥, 仙门, 旅游区, 夏天, 度假]",[王府 竹海 总结 这份 攻略 赶紧 收藏 每次 惊喜 领略 王府 竹海 四季 春夏秋冬 这...


In [10]:
# UMAP可视化
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine',).fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, hide_document_hover=True, hide_annotations=False)

# 使用sentence-transformers模型

In [11]:
# 1. 词向量模型，同时加载本地训练好的词向量
embedding_model = embedding_model = SentenceTransformer(
  'paraphrase-multilingual-mpnet-base-v2',
) # 使用Sentencetransformers模型
embeddings = np.load('../../data/embedding_sen.npy') # 使用sentence-transformers向量
print(embeddings.shape)

# 2. 创建分词模型
vectorizer_model = CountVectorizer() # 因为我们已经分好词了，所以这里不需要传入分词函数了

# 3. 创建UMAP降维模型
umap_model = UMAP(
  n_neighbors=15,
  n_components=5,
  min_dist=0.0,
  metric='cosine',
  random_state=42  # ⚠️ 防止随机 https://maartengr.github.io/BERTopic/faq.html
)

# 4. 创建HDBSCAN聚类模型
# 如果要建设离群值，可以减小下面两个参数
# https://hdbscan.readthedocs.io/en/latest/faq.html
hdbscan_model = HDBSCAN(
  min_cluster_size=40,
  min_samples=40,
)

# 5. 创建CountVectorizer模型
vectorizer_model = CountVectorizer(stop_words=['洛阳', '旅游', '文化'])

# 正式创建BERTopic模型
topic_model = BERTopic(
  embedding_model=embedding_model,
  vectorizer_model=vectorizer_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
)

# 查看主题
topics, probs = topic_model.fit_transform(docs, embeddings=embeddings) #传入训练好的词向量
topic_model.get_topic_info()

(1000, 384)


,Topic,Count,Name,Representation,Representative_Docs
0,-1,427,-1_景区_城市_河南_游客,"[景区, 城市, 河南, 游客, 洛阳市, 中国, 年票, 国家, 发展, 博物馆]",[小洛微 提醒 河南 多家 景区 陆续 发布 开园 公告 台风 杜苏芮 强度 逐渐 减弱 河...
1,0,194,0_中国_石窟_龙门石窟_艺术,"[中国, 石窟, 龙门石窟, 艺术, 河南, 世界, 历史, 博物馆, 文化遗产, 位于]",[河南 旅游 之焦 洛阳 --- 龙门石窟 中国 石刻 艺术 宝库 现为 世界 文化遗产 全...
2,1,187,1_河南_活动_发展_中国,"[河南, 活动, 发展, 中国, 洛阳市, 白云山, 主题, 服务, 历史, 工作]",[行走 河南 读懂 中国 关注 全省 文旅文创 发展 大会 二十大 报告 指出 坚持 以文塑...
3,2,132,2_景区_游客_高速_旅客,"[景区, 游客, 高速, 旅客, 郑州, 门票, 活动, 免费, 假期, 高铁]",[日起 河南 景区 门票 河南省政府 新闻办 召开 新闻 发布会 获悉 豫见 春天 惠游 老...
4,3,60,3_景区_王府_竹海_旅行,"[景区, 王府, 竹海, 旅行, 游客, 位于, 小浪底, 黄河, 仙门, 拍照]",[王府 竹海 总结 这份 攻略 赶紧 收藏 每次 惊喜 领略 王府 竹海 四季 春夏秋冬 这...


In [12]:
# UMAP可视化
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine',).fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, hide_document_hover=True, hide_annotations=False)